In [1]:
import numpy as np
import csv
import os
import math
import matplotlib.pyplot as plt
import pandas
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
import numpy as np

os.chdir('/Users/sreelakshmirajendrakumar/Downloads')



In [2]:
#finding out the number of columns of the csv file
dataset = pandas.read_csv('OP_data2.csv')   
X = dataset.iloc[:, 1: 19975]
y = dataset.iloc[:, 19975] 
#converting the pandas dataframe to array 
x_data=X.to_numpy()
y_data=y.to_numpy()



In [3]:
# ten-fold cross validation
kf=KFold(n_splits=10, random_state=None, shuffle=False)
kf.get_n_splits(X)
x_train=[]
x_test=[]
y_train=[]
y_test=[]
for train_index, test_index in kf.split(X):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = x_data[train_index], x_data[test_index]
    x_train.append(X_train)
    x_test.append(X_test)
    Y_train, Y_test = y_data[train_index], y_data[test_index] 
    y_train.append(Y_train)
    y_test.append(Y_test)    

l_data=np.arange(0,.31,.01) 
d=19974


In [4]:
def multivariate(X_train,Y_train,l):           
    #restructuring the X_test array
    N=len(X_train)
    W=[]
    Z=[]
    for i in range(0,d):
        W.append([])
        Z.append([])
        index_0=[]
        index_1=[]
        for j in range(0,N):
            if Y_train[j]==0:
                W[i].append(X_train[j][i])
                index_0.append(j)
            else:
                Z[i].append(X_train[j][i])
                index_1.append(j)
    w=np.transpose(W)
    z=np.transpose(Z)

    # finding u_(1,0) and U_(1,1)
    u_10=[]
    for i in range(0,d):
        sum=0
        for j in range(0,len(index_0)):
            sum+=w[j][i]
        if len(index_0)!=0:
            sum=sum/len(index_0)
        u_10.append(sum)
    
    
    u_11=[]
    for i in range(0,d):
        sum=0
        for j in range(0,len(index_1)):
            sum+=z[j][i]
        if len(index_1)!=0:
            sum=sum/len(index_1)
        u_11.append(sum)
    
        
    # finding u_1 and u_2
    u_1=[]
    u_2=[]
    for i in range(0,d):
        sum=0
        sum_2=0
        for j in range(0,N):
            sum+=X_train[j][i]
            sum_2+=((X_train[j][i])**2)
        u_1.append(sum/N)
        u_2.append(sum_2/N)

    # apply the equation --> 1. checking the value of lambda and element-wise opeation on the matrices

    N_0=len(index_0)
    N_1=len(index_1)

    condition=[]
    for i in range(0,d):
        c=((N_0*N_1/(N**2))*(abs(u_11[i]-u_10[i])/np.sqrt(u_2[i]-(u_1[i]**2))))
        condition.append(c)
    
    m_0=[]
    m_1=[]
    variance=[]
    sigma=[]
    for i in range(0,d):
        if l>=condition[i]:
            m_0.append(u_1[i])
            m_1.append(u_1[i])
            temp=(u_2[i]-(u_1[i]**2))
            variance.append(temp)
            sigma.append(np.sqrt(temp))
        
        elif l<condition[i]:
            s=u_2[i]-(u_1[i]**2)-((N_0*N_1/N**2)*((u_11[i]-u_10[i])**2))
            sig=abs(u_11[i]-u_10[i])*l+np.sqrt(((u_11[i]-u_10[i])**2)*(l**2)+4*s)
            sigma.append(np.sqrt(sig))
            if u_11[i]>=u_10[i]:
                temp1=u_10[i]+((N_1/N)*sig*l)
                temp2=u_11[i]-((N_0/N)*sig*l)
                m_0.append(temp1)
                m_1.append(temp2)
            else:
                temp3=u_10[i]-((N_1/N)*sig*l)
                temp4=u_11[i]+((N_0/N)*sig*l)
                m_0.append(temp3)
                m_1.append(temp4)
            
    return m_0,m_1,sigma  





In [ ]:
# find m_0,m_1,and sigma corresponding to all the training sets with different lambda values
# this involves x_train, y_train, and lambda values    
#for i in range(0,len(l_data)):
M0=[]
M1=[]
SIGMA=[]
n=0
for lambd in l_data:
    M0.append([])
    M1.append([])
    SIGMA.append([])
    for j in range(0,10):
        M0[n].append([])
        M1[n].append([])
        SIGMA[n].append([])
        M0[n][j],M1[n][j],SIGMA[n][j]=multivariate(x_train[j],y_train[j],lambd)
    n+=1
#======================================================================   

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:61: RuntimeWarning: invalid value encountered in double_scalars


In [ ]:
# find the average m_0,m_1, and sigma values for diferent lambda values
#this involves lambda values, and all the m_0, sigma, and m_1 values
m_0_av=[]
m_1_av=[]
sigma_av=[]
r=0
for lambd in l_data:
    m_0_av.append([])
    m_1_av.append([])
    sigma_av.append([])
    for j in range(0,d):
        a=0
        b=0
        c=0
        for m in range(0,10):
            a+=M0[m][j]
            b+=M1[m][j]
            c+=SIGMA[m][j]
        a=a/10
        b=b/10
        c=c/10
        m_0_av[r].append(a)
        m_1_av[r].apend(b)
        sigma_av[r].append(c)
    r+=1



In [ ]:
# test on the test data using the equation in the question with the help of the m_0,m_1, and sigma values obtained
# this requires the latest parameter values, x_test, y_test, and different lambda values   
# this also requires new values of y to be computed to find the error
Y_new=[]
s=0
for lambd in l_data:
    Y_new.append([])
    for q in range(0,len(y_data)):
        comp1=0
        comp2=0
        for j in range(0,d):
            comp1+=(((x_data[i][q][j]-m_0_av[i][j])**2)/(sigma_av[i][j]**2))
            comp2+=(((x_data[i][q][j]-m_1_av[i][j])**2)/(sigma_av[i][j]**2))
        if comp1<comp2:
            Y_new[s].append(0)
        else:
            Y_new[s].append(1)
    s+=1

eror=[]
w=0
for lamd in l_data:
    error.append([])
    for i in range(0,len(y_data)):
        if Y_new[w][i]!=y_data[w][i]:
            error_rate+=1
        error=error_rate/len(y_data)
    w+=1
# get the error and compare
# this involves y_test, new y values, and lambda